In [1]:
%load_ext autoreload
%autoreload 2

import odc.stac
import pandas as pd
import pystac_client

from pyTMD.compute import tide_elevations
import pandas as pd
import numpy as np


GAUGE_X = 122.2183
GAUGE_Y = -18.0008
ENSEMBLE_MODELS = ["EOT20", "HAMTIDE11"]  # simplified for tests

## Load fixtures

In [2]:
def load_satellite_ds():
    """
    Load a sample timeseries of Landsat 8 data using odc-stac
    """
    # Connect to stac catalogue
    catalog = pystac_client.Client.open("https://explorer.dea.ga.gov.au/stac")

    # Set cloud defaults
    odc.stac.configure_rio(
        cloud_defaults=True,
        aws={"aws_unsigned": True},
    )

    # Build a query with the parameters above
    bbox = [GAUGE_X - 0.08, GAUGE_Y - 0.08, GAUGE_X + 0.08, GAUGE_Y + 0.08]
    query = catalog.search(
        bbox=bbox,
        collections=["ga_ls8c_ard_3"],
        datetime="2020-01/2020-02",
    )

    # Search the STAC catalog for all items matching the query
    ds = odc.stac.load(
        list(query.items()),
        bands=["nbart_red"],
        crs="epsg:3577",
        resolution=30,
        groupby="solar_day",
        bbox=bbox,
        fail_on_error=False,
        chunks={},
    )

    return ds

satellite_ds = load_satellite_ds()

def load_measured_tides_ds():
    """
    Load measured sea level data from the Broome ABSLMP tidal station:
    http://www.bom.gov.au/oceanography/projects/abslmp/data/data.shtml
    """
    # Metadata for Broome ABSLMP tidal station:
    # http://www.bom.gov.au/oceanography/projects/abslmp/data/data.shtml
    ahd_offset = -5.322

    # Load measured tides from ABSLMP tide gauge data
    measured_tides_df = pd.read_csv(
        "../tests/data/IDO71013_2020.csv",
        index_col=0,
        parse_dates=True,
        na_values=-9999,
    )[["Sea Level"]]

    # Update index and column names
    measured_tides_df.index.name = "time"
    measured_tides_df.columns = ["tide_height"]

    # Apply station AHD offset
    measured_tides_df += ahd_offset

    # Return as xarray dataset
    return measured_tides_df.to_xarray()

satellite_ds = load_satellite_ds()
measured_tides_ds = load_measured_tides_ds()

## Testing pyTMD

In [47]:
from eo_tides import model_tides

x, y, crs, method, model = GAUGE_X, GAUGE_Y, "EPSG:4326", "spline", "EOT20"
x, y, crs, method, model = GAUGE_X, GAUGE_Y, "EPSG:4326", "bilinear", "EOT20"
x, y, crs, method, model = -1034913, -1961916, "EPSG:3577", "bilinear", "EOT20"


# Run EOT20 tidal model for locations and timesteps in tide gauge data
modelled_tides_df = model_tides(
    x=[x],
    y=[y],
    time=measured_tides_ds.time,
    crs=crs,
    method=method,
    directory="../tests/data/tide_models",
)

# Run equivalent pyTMD code to verify same results
pytmd_tides = tide_elevations(
        x=x, 
        y=y, 
        delta_time=measured_tides_ds.time,
        DIRECTORY="../tests/data/tide_models",
        MODEL="EOT20",
        EPSG=int(crs[-4:]),
        TIME="datetime",
        EXTRAPOLATE=True,
        CUTOFF=np.inf,
        METHOD=method,
        # CORRECTIONS: str | None = None,
        # INFER_MINOR: bool = True,
        # MINOR_CONSTITUENTS: list | None = None,
        # APPLY_FLEXURE: bool = False,
        # FILL_VALUE: float = np.nan
        )

np.allclose(modelled_tides_df.tide_height.values, pytmd_tides.data)

Modelling tides using EOT20


True

### Error for out of bounds

In [10]:
from eo_tides import model_tides

x, y = 180, -50


# Run EOT20 tidal model for locations and timesteps in tide gauge data
modelled_tides_df = model_tides(
    x=[x],
    y=[y],
    model=["EOT20", "GOT5.5"],
    time=measured_tides_ds.time,
    directory="../tests/data/tide_models",
)

### Modelling ebb and flow tidal phases
The `tag_tides` function also allows us to determine whether each satellite observation was taken while the tide was rising/incoming (flow tide) or falling/outgoing (ebb tide) by setting `ebb_flow=True`. This is achieved by comparing tide heights 15 minutes before and after the observed satellite observation.

Ebb and flow data can provide valuable contextual information for interpreting satellite imagery, particularly in tidal flat or mangrove forest environments where water may remain in the landscape for considerable time after the tidal peak.

Once you run the cell below, our data will now also contain a new `ebb_flow` variable under **Data variables**:

In [ ]:
# Model tide heights
ds = tag_tides(
    ds, 
    ebb_flow=True,     
    directory="../../tests/data/tide_models",
)

# Print output data
print(ds)

Setting tide modelling location from dataset centroid: 122.27, -18.09
Modelling tides using EOT20
Modelling tidal phase (e.g. ebb or flow)
Modelling tides using EOT20
<xarray.Dataset> Size: 688MB
Dimensions:      (y: 1185, x: 1099, time: 44)
Coordinates:
  * y            (y) float64 9kB 8.017e+06 8.017e+06 ... 7.982e+06 7.982e+06
  * x            (x) float64 9kB 4.068e+05 4.068e+05 ... 4.397e+05 4.398e+05
    spatial_ref  int32 4B 32751
  * time         (time) datetime64[ns] 352B 2024-01-07T01:55:31.679580 ... 20...
Data variables:
    nbart_red    (time, y, x) float32 229MB dask.array<chunksize=(1, 1185, 1099), meta=np.ndarray>
    nbart_green  (time, y, x) float32 229MB dask.array<chunksize=(1, 1185, 1099), meta=np.ndarray>
    nbart_blue   (time, y, x) float32 229MB dask.array<chunksize=(1, 1185, 1099), meta=np.ndarray>
    tide_height  (time) float32 176B -0.2511 0.5276 1.517 ... 0.7157 -0.03352
    ebb_flow     (time) object 352B 'Ebb' 'Ebb' 'Flow' ... 'Flow' 'Flow' 'Ebb'


We now have data giving us the both the tide height and tidal phase ("ebb" or "flow") for every satellite image:

In [ ]:
ds[["time", "tide_height", "ebb_flow"]].drop_vars("spatial_ref").to_dataframe().head()

,tide_height,ebb_flow
time,,
2024-01-07 01:55:31.679580,-0.251089,Ebb
2024-01-08 01:49:36.716728,0.527567,Ebb
2024-02-08 01:56:00.413098,1.517171,Flow
2024-02-09 01:49:37.113806,2.011433,Flow
2024-03-20 01:49:39.697193,0.368091,Flow


We could for example use this data to filter our observations to keep ebbing phase observations only:

In [ ]:
ds_ebb = ds.where(ds.ebb_flow == "Ebb", drop=True)
print(ds_ebb)

<xarray.Dataset> Size: 219MB
Dimensions:      (time: 14, y: 1185, x: 1099)
Coordinates:
  * y            (y) float64 9kB 8.017e+06 8.017e+06 ... 7.982e+06 7.982e+06
  * x            (x) float64 9kB 4.068e+05 4.068e+05 ... 4.397e+05 4.398e+05
    spatial_ref  int32 4B 32751
  * time         (time) datetime64[ns] 112B 2024-01-07T01:55:31.679580 ... 20...
Data variables:
    nbart_red    (time, y, x) float32 73MB dask.array<chunksize=(1, 1185, 1099), meta=np.ndarray>
    nbart_green  (time, y, x) float32 73MB dask.array<chunksize=(1, 1185, 1099), meta=np.ndarray>
    nbart_blue   (time, y, x) float32 73MB dask.array<chunksize=(1, 1185, 1099), meta=np.ndarray>
    tide_height  (time) float32 56B -0.2511 0.5276 1.753 ... 0.6212 -0.03352
    ebb_flow     (time) object 112B 'Ebb' 'Ebb' 'Ebb' ... 'Ebb' 'Ebb' 'Ebb'
